In [ ]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

## 데이터 가져오기 

In [ ]:
train_data= pd.read_table('ratings_train.txt')
test_data= pd.read_table('ratings_test.txt')

## 데이터 살펴보기 

In [ ]:
print(len(train_data)) # 학습시킬 데이터 개수

In [ ]:
train_data[:5] # 상위 5개 출력

In [ ]:
print(len(test_data)) # 평가를 위한 데이터 개수

In [ ]:
test_data[:5] # 상위 5개 출력

In [ ]:
train_data['label'].value_counts().plot(kind='bar') # 학습데이터에서 label 값의 분포

In [ ]:
train_data.groupby('label').size().reset_index(name='count') # 위 내용을 테이블로 보기

## Null 값 지우기 

In [ ]:
train_data.isnull().values.any() # Null 값이 있는지 확인 (결과: True/False)

In [ ]:
train_data.loc[train_data.document.isnull()] # Null 값이 있는 레코드 출력

In [ ]:
train_data=train_data.dropna(how='any') # Null 값이 존재하는 행 제거

In [ ]:
train_data.isnull().values.any() # Null 값이 있는지 확인 (결과: True/False)

In [ ]:
len(train_data) # Null 값이 있던 5개 레코드가 지우졌는지 확인

## 전처리 수행하기 

알파벳과 공백을 제외하고 모두 제거(구두점이나 특수문자 제거)

**예시1** 영문에서 알파벳과 공백만 남기기

In [ ]:
text='do!!! you expect... people~ to~ read~ the FAQ, etc. and actually accept hard~! atheism?@@'
re.sub(r'[^a-zA-Z ]', '', text) #알파벳과 공백을 제외하고 모두 제거

**예시2** 한글에서 한글과 공백만 남기기

  * ㄱ-ㅎ: 3131-314E
  * ㅏ-ㅣ: 314F-3163
  * 가-힣: AC00-D7A3

UNICODE 표 참고

자음모음(ㄱ-ㅎ, ㅏ-ㅣ):https://www.unicode.org/charts/PDF/U3130.pdf

완성형 한글(가-힣): https://www.unicode.org/charts/PDF/UAC00.pdf

In [ ]:
text="아 더빙.. 진짜 짜증나네요 목소리"
text.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]","") #알파벳과 공백을 제외하고 모두 제거


In [ ]:
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# 한글과 공백을 제외하고 모두 제거
train_data[:5]

## 토큰화와 불용어 제거 

In [ ]:
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다'] # 불용어 정의 (지속적으로 검토하여 정의할 수 있음)

토큰화를 위해 형태소분석기 KoNLPy의 Okt를 사용

`stem=True`는 일부 단어를 정규화 해준다. 예를 들어 '이런'이 '이렇다'로 변환되고, '만드는'이 만들다로 변환된다.

In [ ]:
okt = Okt()
okt.morphs('와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔', stem=True)

In [ ]:
X_train=[]
for sentence in train_data['document']:
    temp_X = []
    temp_X=okt.morphs(sentence, stem=True) # 토큰화
    temp_X=[word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

In [ ]:
print(X_train[:3]) # 상위 3개 출력

In [ ]:
# 평가데이터에도 동일하게 적용
test_data=test_data.dropna(how='any') # Null 값 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행

X_test=[]
for sentence in test_data['document']:
    temp_X = []
    temp_X=okt.morphs(sentence, stem=True) # 토큰화
    temp_X=[word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test.append(temp_X)

## 정수 인코딩 진행

각 샘플 데이터에 단어 데신 인덱트를 부여함

In [ ]:
max_words = 35000
tokenizer = Tokenizer(num_words=max_words) # 상위 35,000개의 단어만 보존
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
print(X_train[:3]) # 상위 3개만 보기

리뷰의 길이 살펴보기

 * 가장 긴 리뷰는 69
 * 평균적으로 10 정도의 길이를 가짐

In [ ]:
# len()은 list에서 element의 개수
print('리뷰의 최대 길이 :',max(len(l) for l in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(s) for s in X_train], bins=50)
plt.xlabel('length of Data')
plt.ylabel('number of Data')
plt.show()

모델이 처리할 수 있도록 X_train과 X_test의 길기를 **30**으로 맞춘다.

In [ ]:
max_len=30
# 전체 데이터의 길이는 30으로 맞춘다.
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
X_train[0] # 변경된 데이터 살펴보기

label을 별도로 y_train과 y_test에 저장

In [ ]:
y_train=np.array(train_data['label'])
y_test=np.array(test_data['label'])
print(y_train,y_test)

In [ ]:
y_train

## LSTM으로 영화리뷰 감성 분류하기

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Embedding(max_words, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=4, batch_size=60, validation_split=0.2)

테스트 정확도 측정

In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

In [ ]:
predict_in=X_train[0]
predictation_single=model.predict(predict_in)
print(predictation_single)